## Урок 12. Модель Transformer-2

### Задание

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [3]:
import pandas as pd

import json
import random

import re

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [5]:
test_records = read_gazeta_records("gazeta_test.txt")

In [6]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [7]:
def summurize(n):

  article_text = ("{}".format(test_records[n]["text"]))

  input_ids = tokenizer(
      [WHITESPACE_HANDLER(article_text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]

  output_ids = model.generate(
      input_ids=input_ids,
      max_length=84,
      no_repeat_ngram_size=2,
      num_beams=4
  )[0]

  summary = tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )

  print('TEXT:   ', article_text)
  print('SUMMARY:', summary)

In [8]:
summurize(0)

TEXT:    Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, как они и им подобные объекты образовались в космосе». Каждый проект из

In [9]:
summurize(3)

TEXT:    Народная артистка РСФСР Надежда Бабкина в интервью шоу «Секрет на миллион» Леры Кудрявцевой рассказала о взаимоотношениях со своим гражданским мужем Евгением Гором (настоящая фамилия — Горшечков), который младше певицы на 30 лет. По словам исполнительницы, которая в этом марте отметит 70-летие, в своей квартире в центре Москвы она живет на постоянной основе без любимого мужчины, лишь периодически встречаясь там со своим избранником. «Когда я здесь одна, мне говорят: «Тебе там скукота». Никогда! Я иногда просто умышленно хочу побыть одна. Могу голая здесь ходить. Конечно, свет погашу. Чтобы вдруг из Кремля с подзорной трубы не подглядывали. А вдруг — понравлюсь?» — пояснила Бабкина. Она также призналась, что не обращает внимания на сплетни, касающиеся ее отношений с Гором, с которым познакомилась еще в начале нулевых, тогда как ее мужчина переживает по поводу слухов, считая, что люди видят в ее романе с Бабкиной расчет. «Эта болезненная тема не для меня. Как нас только не пенял

In [10]:
summurize(15)

TEXT:    Росстандарт готовит арсенал для проверок автозаправочных станций на недолив топлива: проверяться будет не конкретная колонка, а ее работа как системы, в том числе ее связь с оператором за кассой. Контролеров при работе методом тайного покупателя вооружат скрытым прибором, который сможет «ловить» недолив на уровне 1%. Кроме того, новый ГОСТ защитит топливораздаточные колонки от приспособлений и вредоносного ПО, обманывающих расходомер. «Российская газета» пообщалась с главой Росстандарта Алексеем Абрамовым и выяснила, что в будущем с недоливом топлива на автозаправочных станциях будут бороться комплексно, используя большой арсенал инструментов. По словам Абрамова, недолив случается преимущественно из-за вторжения в программное обеспечение топливораздаточных колонок (ТРК). «Программа может быть настроена так, что на любом пистолете устанавливается конкретный процент недолива… Индикатор покажет вам 30 литров, но на самом деле в бак попало 29,7 литра, а если показывает 60 литров —

In [11]:
summurize(77)

TEXT:    Сценарист и писатель Михаил Идов сообщил на своей странице в фейсбуке о смерти режиссера фильмов «Чародеи» и «Приключения Электроника» Константина Бромберга. «Два дня назад в Детройте скончался режиссер Константин Бромберг, постановщик советских мегахитов «Чародеи» и «Приключения Электроника». Не видел пока ни одного сообщения об этом в прессе — ну вот напишу хоть тут; как-никак часть истории российского кино», — написал Идов. Его родители часто общались с Бромбергом, они поддерживали контакт до последних дней режиссера. «Сам я его видел только пару раз в девяностых, когда еще жил у них в Детройте. В США Бромберг ничем связанным с кино не занимался и, кажется, даже не пытался, просто жил», — рассказал Идов. По его словам, Бромберг какое-то время «грезил удивительными планами» о создании передвижного музея, в котором были бы выставлены копии мировых художественных шедевров. Идов отметил, что «только сейчас понимает», потенциал этой идеи, наблюдая «мультимедийные выставки с виде

In [12]:
summurize(100)

TEXT:    Фрегаты Черноморского флота «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы. Они следуют из Севастополя в дальнюю морскую зону, где войдут в состав сил постоянной группировки ВМФ в Средиземном море, передает «Интерфакс» со ссылкой на официального представителя ЧФ Алексея Рулева. Он рассказал, что при переходе экипажи обоих кораблей провели учения на полигонах боевой подготовки флота в Черном море. Как напомнил Рулев , третий фрегат ЧФ этой серии — «Адмирал Эссен» — выполняет задачи в Средиземном море уже с декабря 2019 года. Корабли Черноморского флота относятся к новой серии фрегатов проекта 11356Р/М (также используется обозначение 11357). Эти многоцелевые фрегаты-сторожевики дальней морской зоны предназначены для уничтожения надводных кораблей и судов, подводных лодок и наземных объектов противника. Они также используются для дозорной службы, патрулирования и охраны морских коммуникаций. На борту ф

 **Вывод:** качество суммаризации довольно высокое.